In [ ]:

#  Here we demonstrate the following Dask DataFrame technique;
#
#     .  Given a source DataFrame, create a second new DataFrame with just unique
#        values, and the ability to link back to the source.
#
#        We might use this technique when given a flat file of data, that we derive
#        nodes and edges from.


In [16]:

import numpy as np
import pandas as pd
   #
import dask.array as da
import dask.dataframe as dd

from tabulate import tabulate

print("--")


--


In [19]:

#  Our source DataFrame

np_airports = np.array([
       #
   ["MKE", "Milwaukee"     , "WI", "Airport"],
   ["ORD", "Chicago O-Hare", "IL", "Airport"],
   ["SJC", "San Jose"      , "CA", "Airport"],
   ["LAX", "Los Angeles"   , "CA", "Airport"],
   ["DEN", "Denver"        , "CO", "Airport"],
       #
   ], dtype="str")

pd_airports = pd.DataFrame(np_airports, columns = ["airport_code", "airport_name", "state_code", "LABEL"])





# df_airports = dd.io.from_dask_array(pd_airports)



# my_array = np.array([[11,22,33],[44,55,66]])
# 
# df = pd.DataFrame(my_array, columns = ['Column_A','Column_B','Column_C'])



#  df_airports = dd.io.from_dask_array([
#         #
#     ["MKE", "Milwaukee"          ,"WI", "Airport"],
#     ["ORD", "Chicago O-Hare"     ,"IL", "Airport"],
#     ["SJC", "San Jose"           ,"CA", "Airport"],
#     ["LAX", "Los Angeles"        ,"CA", "Airport"],
#     ["DEN", "Denver"             ,"CO", "Airport"],
#         #
#     ], columns = ["airport_code", "airport_name", "state_code", "LABEL"])
#  
# print(tabulate(df_airports, headers='keys', tablefmt='psql'))


# pd.array(['a', 'b'], dtype=str)



# x = da.ones((4, 2), chunks=(2, 2))
# 
# df = dd.io.from_dask_array(x, columns=['a', 'b'])
# 
# df.compute()


